In [267]:
# import libraries
#!pip install BeautifulSoup4
#!pip install urllib3
#import urllib
#import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
from datetime import date 
from random import randint

### Pass cookies to be used to authenticate logon at site

In [159]:
df_cookies = pd.read_csv(r'C:\Users\stefa\Documents\GitHub\first_scraper-master\cookies.csv',header=None).dropna()
names = list(df_cookies[0])
values = list(df_cookies[1])
cookies = dict(zip(names, values))
#cookies
#df_cookies.set_index(df_cookies[0])#,drop=True, inplace=True)#.to_dict()

### List of addresses

In [319]:
# Add count to end URL, 0 for first 25, 25 for next 25, 50... 
season_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AS_2018&myteam=0&sort=AR&sdir=1&count='
day30_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AL30&myteam=0&sort=AR&sdir=1&count='
day14_avg_url = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=S_AL14&myteam=0&sort=AR&sdir=1&count='
opponents = 'https://basketball.fantasysports.yahoo.com/nba/14392/players?status=ALL&pos=P&cut_type=33&stat1=O_O&myteam=0&sort=AR&sdir=1&count='

In [386]:
def pull_25_opponents(url,ranks):
    r = requests.post(str(url+ranks), cookies=cookies)
    soup = BeautifulSoup(r.text,'lxml')
    table = soup.find('table',{'class':'Table Ta-start Fz-xs Table-mid Table-px-xs Table-interactive'})
    mydivs = table.findAll('div', {"class": "ysf-player-name Nowrap Grid-u Relative Lh-xs Ta-start"})
    player_list = []
    for player in mydivs:
        player_list.append(player.text)
    df_player = pd.DataFrame(player_list)   
    dummy_array = np.empty((27,24))
    dummy_array[:] = np.nan
    player_array = pd.DataFrame(dummy_array)
    row_marker=0
    for row in table.findAll('tr'):
        column_marker = 0
        for data in row.findAll('td'):
            player_array.iloc[row_marker,column_marker] = data.get_text()
            column_marker += 1
        row_marker+=1
    player_array = player_array.dropna()
    columns_we_like = [10,11,12,13,14,15,16]
    player_array = player_array[columns_we_like].reset_index(drop=True)
    df_player = df_player.merge(player_array, left_index =True, right_index=True)
    df_player['games_this_week']= 7 - (np.where(df_player[[10]]=='',1,0)+np.where(df_player[[11]]=='',1,0)+ np.where(df_player[[12]]=='',1,0)+ np.where(df_player[[13]]=='',1,0)+ np.where(df_player[[14]]=='',1,0)+ np.where(df_player[[15]]=='',1,0)+ np.where(df_player[[16]]=='',1,0))

    return df_player

### Pull function
- Pulls based on specified URL request
- Cookies used to establish logon session from cookies.csv
- Creates player array for 25 players at a time
- Loop to create full datasets

In [388]:
def pull_25_opponents(url,ranks):
    r = requests.post(str(url+ranks), cookies=cookies)
    soup = BeautifulSoup(r.text,'lxml')
    table = soup.find('table',{'class':'Table Ta-start Fz-xs Table-mid Table-px-xs Table-interactive'})
    mydivs = table.findAll('div', {"class": "ysf-player-name Nowrap Grid-u Relative Lh-xs Ta-start"})
    player_list = []
    for player in mydivs:
        player_list.append(player.text)
    df_player = pd.DataFrame(player_list)   
    dummy_array = np.empty((27,24))
    dummy_array[:] = np.nan
    player_array = pd.DataFrame(dummy_array)
    row_marker=0
    for row in table.findAll('tr'):
        column_marker = 0
        for data in row.findAll('td'):
            player_array.iloc[row_marker,column_marker] = data.get_text()
            column_marker += 1
        row_marker+=1
    player_array = player_array.dropna()
    columns_we_like = [10,11,12,13,14,15,16]
    player_array = player_array[columns_we_like].reset_index(drop=True)
    df_player = df_player.merge(player_array, left_index =True, right_index=True)
    df_player['games_this_week']= 7 - (np.where(df_player[[10]]=='',1,0)+np.where(df_player[[11]]=='',1,0)+ np.where(df_player[[12]]=='',1,0)+ np.where(df_player[[13]]=='',1,0)+ np.where(df_player[[14]]=='',1,0)+ np.where(df_player[[15]]=='',1,0)+ np.where(df_player[[16]]=='',1,0))
    df_player['name'] = df_player[[0]]
    return df_player[['name','games_this_week']]

In [204]:
def pull_25(url,ranks):
    r = requests.post(str(url+ranks), cookies=cookies)
    soup = BeautifulSoup(r.text,'lxml')
    table = soup.find('table',{'class':'Table Ta-start Fz-xs Table-mid Table-px-xs Table-interactive'})
    mydivs = table.findAll('div', {"class": "ysf-player-name Nowrap Grid-u Relative Lh-xs Ta-start"})
    player_list = []
    for player in mydivs:
        player_list.append(player.text)
    df_player = pd.DataFrame(player_list)   
    dummy_array = np.empty((27,23))
    dummy_array[:] = np.nan
    player_array = pd.DataFrame(dummy_array)
    row_marker=0
    for row in table.findAll('tr'):
        column_marker = 0
        for data in row.findAll('td'):
            player_array.iloc[row_marker,column_marker] = data.get_text()
            column_marker += 1
        row_marker+=1
    player_array['INJ'] = np.where(player_array[1].str.contains("INJ"),"INJ",0)
    player_array = player_array.dropna()
    columns_we_like = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,'INJ']
    player_array = player_array[columns_we_like].reset_index(drop=True)
    df_player = df_player.merge(player_array, left_index =True, right_index=True)
    df_player.columns=['Player','Owner','GP','O_Rank','C_Rank','Owned','MP','FGs','FG_pct','FTs','FT_pct',
                       '3pt','Pts','Rbs','Ast','Stl','Blk','TO','INJ']
    return df_player

### Build top 250

In [311]:
def build_250_player(url_pull):
    df_player = pull_25(url_pull,str(0))
    for x in range(25,250,25):
        df_temp = pull_25(url_pull,str(x))
        df_player = df_player.append(df_temp)
        time.sleep(randint(50,100)/10) 
    df_player = df_player.reset_index(drop=True)
    df_player['pos'] = (df_player['Player'].str.split(pat='-', n=1, expand=True))[1]
    df_player[['Player','pos','Owner','INJ','GP','O_Rank','C_Rank','Owned','MP','FGs','FG_pct','FTs','FT_pct',
                           '3pt','Pts','Rbs','Ast','Stl','Blk','TO']]
    return df_player


In [397]:
def build_opponents(url_pull):
    df_player = pull_25_opponents(url_pull,str(0))
    for x in range(25,350,25):
        df_temp = pull_25_opponents(url_pull,str(x))
        df_player = df_player.append(df_temp)
        time.sleep(randint(50,100)/10) 
    df_player = df_player.reset_index(drop=True)
    df_player[['name','games_this_week']]
    return df_player    

In [398]:
#df_7 = build_250_player(day7_avg_url)
#df_14 = build_250_player(day14_avg_url)
#df_30 = build_250_player(day30_avg_url)
df_opponents = build_opponents(opponents)

In [395]:
df_14

,Player,Owner,GP,O_Rank,C_Rank,Owned,MP,FGs,FG_pct,FTs,FT_pct,3pt,Pts,Rbs,Ast,Stl,Blk,TO,INJ,pos
0,"K. Leonard Tor - SG,SF",Westbrook 4 Kidd,3,13,1,100%,40:00,12.0/22.3,.537,8.7/9.0,.963,2.7,35.3,7.3,4.0,3.0,0.7,3.3,0,"SG,SF"
1,"J. Harden Hou - PG,SG",Green Machine,7,1,2,100%,39:17,13.7/31.1,.440,15.6/17.1,.908,4.7,47.7,8.4,5.7,2.4,0.9,4.9,0,"PG,SG"
2,"S. Curry GS - PG,SG",Taxmaddness,7,4,3,100%,34:00,10.3/20.3,.507,5.1/5.1,1.000,5.9,31.6,4.9,4.9,0.6,0.6,1.4,0,"PG,SG"
3,"A. Davis NO - PF,C",You Reach I Teach,3,2,4,100%,38:40,12.0/25.7,.468,9.3/10.7,.875,1.0,34.3,13.7,4.3,1.3,1.7,1.3,INJ,"PF,C"
4,"P. George OKC - SG,SF,PF",Sometimes I Dribble,6,12,5,100%,38:00,8.3/19.0,.439,7.5/8.5,.882,4.5,28.7,7.3,5.2,3.0,0.3,2.0,0,"SG,SF,PF"
5,"K. Irving Bos - PG,SG",evan's Neat Team,5,18,6,100%,37:00,11.8/21.0,.562,3.8/4.6,.826,3.6,31.0,4.8,10.8,2.4,0.6,4.2,0,"PG,SG"
6,R. Rondo LAL - PG,FA,1,184,7,67%,37:00,6.0/11.0,.545,0.0/0.0,-,3.0,15.0,6.0,13.0,1.0,2.0,2.0,0,PG
7,K. Towns Min - C,14 is the new 24,6,5,8,100%,30:00,8.7/16.8,.515,6.0/6.3,.947,1.8,25.2,10.2,3.8,1.7,1.2,2.5,0,C
8,R. Gobert Uta - C,You Reach I Teach,6,27,9,100%,33:49,6.2/10.3,.597,4.8/6.3,.763,0.0,17.2,16.8,2.8,0.8,3.2,1.5,0,C
9,"N. Jokic Den - PF,C",Sometimes I Dribble,6,8,10,100%,30:40,9.5/18.0,.528,5.0/5.7,.882,1.7,25.7,12.0,9.2,0.8,0.3,2.2,0,"PF,C"


In [393]:
df_opponents

,name,games_this_week
0,"A. Davis NO - PF,C",3
1,"J. Harden Hou - PG,SG",3
2,"K. Durant GS - SF,PF",3
3,"P. George OKC - SG,SF,PF",3
4,K. Towns Min - C,2
5,"N. Jokic Den - PF,C",4
6,D. Lillard Por - PG,1
7,"N. Vucevic Orl - PF,C",3
8,"S. Curry GS - PG,SG",3
9,"K. Irving Bos - PG,SG",4


In [400]:
df_14 = df_14.merge(df_opponents, how='left', left_on='Player', right_on='name')
df_14 = df_14.fillna(3)

### Export data, analyze this elsewhere!

In [401]:
#df_7.head(2)
df_14.head(2)
#df_30.head(2)


,Player,Owner,GP,O_Rank,C_Rank,Owned,MP,FGs,FG_pct,FTs,...,Pts,Rbs,Ast,Stl,Blk,TO,INJ,pos,name,games_this_week
0,"K. Leonard Tor - SG,SF",Westbrook 4 Kidd,3,13,1,100%,40:00,12.0/22.3,.537,8.7/9.0,...,35.3,7.3,4.0,3.0,0.7,3.3,0,"SG,SF","K. Leonard Tor - SG,SF",2.0
1,"J. Harden Hou - PG,SG",Green Machine,7,1,2,100%,39:17,13.7/31.1,.440,15.6/17.1,...,47.7,8.4,5.7,2.4,0.9,4.9,0,"PG,SG","J. Harden Hou - PG,SG",3.0


In [402]:
path = r'C:\Users\stefa\Documents\GitHub\first_scraper-master\Data'
today = str(date.today())

#df_7.to_csv(path+"_"+today+'_avg_7.csv',encoding='utf-8')
df_14.to_csv(path+"_"+today+'_avg_14.csv',encoding='utf-8')
#df_30.to_csv(path+"_"+today+'_avg_30.csv',encoding='utf-8')

